# Cvičení 2: Drift trosek na hladině

Motivace: Představte si lehký předmět na hladině, který je unášen rychlostí prostředí (např. vítr nebo proud). V tomto cvičení budeme modelovat jeho pohyb v rovině a postupně vybudujeme numerické řešení.

Cíle:
- formálně popsat polohu $\mathbf{r}(t)=(x(t),y(t))$, rychlost $\mathbf{v}(t)$ a zrychlení $\mathbf{a}(t)$ ve 2D,
- sestavit Newtonovu rovnici s odporem vůči prostředí,
- chápat rychlost prostředí jako vektorové pole $\mathbf{w}(\mathbf{r}, t)$ [m/s],
- převést relativní rychlost $\mathbf{v}-\mathbf{w}$ na odporovou sílu,
- použít Eulerovu metodu pro rychlost a lichoběžník pro polohu,
- interpretovat význam parametrů m a c a ověřit numeriku analyticky pro konstantní $\mathbf{w}$.

Zadání: od startu (x0, y0) v čase t0 se částice s nulovou počáteční rychlostí v0 = (0, 0) pohybuje v 2D v prostředí s rychlostí $\mathbf{w}(\mathbf{r}, t)$. Jediná síla je odpor úměrný relativní rychlosti, takže pohyb je plně dán polem $\mathbf{w}$ a parametry m, c. Výstupem bude trajektorie do času t1.

Poznámka: budeme postupně budovat řešení v několika úkolech.


---


## Rychlost prostředí jako časově proměnné vektorové pole

Rychlost prostředí (např. vítr nebo proud) budeme chápat jako vektorové pole, které každému bodu přiřadí vektor rychlosti [m/s]:
$$
\mathbf{w}(\mathbf{r}, t) = (w_x(\mathbf{r}, t), w_y(\mathbf{r}, t)).
$$
Nehomogenní pole znamená, že se vektor liší místo od místa; časová proměnnost znamená, že se pole mění v čase (např. průchod fronty).

Představte si oblast 100 x 100 km rozdělenou na mřížku: v každém bodě známe vektor $\mathbf{w}$ a takový "snapshot" se opakuje třeba po dnech během 30 dní.

Následující dva satelitní snímky (měsíční průměry) ukazují skutečná data rychlosti větru nad oceánem; barvy reprezentují velikost a šipky směr vektorů rychlosti.

![Měsíční průměrné větry, listopad 2014](images/rapidscat_nov2014.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*

![Měsíční průměrné větry, listopad 2015](images/rapidscat_nov2015.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*


---


## Okamžitá poloha a okamžitá rychlost

Stav částice popisujeme vektorem polohy v rovině
$$
\mathbf{r}(t) = (x(t), y(t)).
$$
Okamžitá rychlost a zrychlení jsou derivace podle času:
$$
\mathbf{v}(t) = \mathbf{r}'(t), \qquad \mathbf{a}(t) = \mathbf{v}'(t).
$$
Fyzikálně to znamená, že směr a velikost $\mathbf{v}$ určují, kam a jak rychle se částice pohybuje, zatímco $\mathbf{a}$ popisuje změnu rychlosti. Na obrázcích níže je jednoduchý 2D příklad s konstantním zrychlením; všechny grafy jsou konzistentní a vycházejí ze stejného pohybu.

![Trajektorie v rovině x–y](images/kinematika_trajektorie.png)

![x(t)](images/kinematika_x_t.png)
![y(t)](images/kinematika_y_t.png)

![v_x(t)](images/kinematika_vx_t.png)
![v_y(t)](images/kinematika_vy_t.png)

![a_x(t)](images/kinematika_ax_t.png)
![a_y(t)](images/kinematika_ay_t.png)


---


## Numerický výpočet trajektorie ze znalosti rychlosti

Pokud známe rychlost v čase, poloha je dána integrálem
$$
\mathbf{r}(t)=\mathbf{r}_0+\int_{t_0}^{t}\mathbf{v}(\tau)\,d\tau.
$$
V numerice přecházíme k diskrétním časům a integrál nahradíme lichoběžníkovým pravidlem. Fyzikálně jde o „průměrnou rychlost“ v kroku.

![Lichoběžníkové pravidlo](images/lichobeznikove_pravidlo.png)


---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def vykresli_drift(t, r, v=None, title=None):
    """
    Vykresli trajektorii v rovině a časové průběhy.

    Parametry:
        t     : 1D pole času [s]
        r     : pole tvaru (n, 2) s polohou [m]
        v     : pole tvaru (n, 2) s rychlostí [m/s] (volitelné)
        title : nadpis grafu trajektorie (volitelné)
    """
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.plot(r[:, 0], r[:, 1], label="trajektorie")
    ax.scatter(r[0, 0], r[0, 1], color="green", label="start")
    ax.scatter(r[-1, 0], r[-1, 1], color="red", label="konec")
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    ax.set_aspect("equal", "box")
    if title:
        ax.set_title(title)
    ax.grid(True)
    ax.legend()

    nrows = 2 + (v is not None)
    fig2, axes = plt.subplots(nrows=nrows, ncols=1, sharex=True, figsize=(6, 2.5 * nrows))
    axes = np.atleast_1d(axes)

    axes[0].plot(t, r[:, 0])
    axes[0].set_ylabel("x [m]")
    axes[0].grid(True)

    axes[1].plot(t, r[:, 1])
    axes[1].set_ylabel("y [m]")
    axes[1].grid(True)

    if v is not None:
        speed = np.linalg.norm(v, axis=1)
        axes[2].plot(t, speed)
        axes[2].set_ylabel("|v| [m/s]")
        axes[2].grid(True)

    axes[-1].set_xlabel("t [s]")
    plt.tight_layout()
    plt.show()


---


## Úkol 1: Trajektorie z dané rychlosti

V tomto úkolu předpokládáme známou rychlost v čase a chceme spočítat trajektorii. Využijeme lichoběžníkové pravidlo a prealokaci polí (je to rychlejší a přehlednější).

Zadání: zvolte konstantní rychlost v = (1, 0.3) v intervalu t ∈ [0, 10] s krokem dt a spočtěte polohu.


In [ ]:
def poloha_2d(r0, v, dt):
    """
    Numericky integruje polohu z rychlosti lichoběžníkovým pravidlem.

    Parametry:
        r0 : počáteční poloha [m], pole tvaru (2,)
        v  : rychlost v čase, pole tvaru (n, 2) [m/s]
        dt : krok času [s]

    Návrat:
        r : poloha v čase, pole tvaru (n, 2) [m]
    """
    v = np.asarray(v, dtype=float)
    n = v.shape[0]
    r = np.zeros((n, 2), dtype=float)
    r[0] = np.asarray(r0, dtype=float)

    ###### doplňte kód zde ######
    for i in range(1, n):
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
    #############################

    return r


# Zadání pro test
r0 = np.array([0.0, 0.0])
t0, t1, dt = 0.0, 10.0, 0.1
t = np.arange(t0, t1 + dt, dt)

v_const = np.tile(np.array([1.0, 0.3]), (t.size, 1))
r = poloha_2d(r0, v_const, dt)

vykresli_drift(t, r, v_const, title="Úkol 1: trajektorie z konstantní rychlosti")


---


## Síla jako motivátor pohybu

Navážeme na kinematiku a integraci polohy a přidáme fyzikální příčinu pohybu: odpor vůči prostředí. Působí pouze odporová síla, která závisí na relativní rychlosti částice vůči prostředí.

Odporová síla:
$$
\mathbf{F}_{\mathrm{odpor}} = -c\,(\mathbf{v} - \mathbf{w}(\mathbf{r}, t)).
$$
Newtonův zákon má tvar
$$
 m\,\mathbf{v}'(t) = -c\,(\mathbf{v}(t) - \mathbf{w}(\mathbf{r}, t)).
$$
Z toho plyne
$$
\mathbf{a}(t) = \frac{c}{m}\,(\mathbf{w}(\mathbf{r}, t) - \mathbf{v}(t)).
$$
Jednotky jsou: m [kg], c [kg/s], $\mathbf{w}$ [m/s], $\mathbf{r}$ [m], $\mathbf{v}$ [m/s]. Parametr
$$
\tau = \frac{m}{c}
$$
udává časovou konstantu – tedy jak rychle se systém „přizpůsobí“ lokální rychlosti prostředí $\mathbf{w}$.

Abychom z modelu zrychlení získali rychlost $\mathbf{v}(t)$, potřebujeme numericky integrovat zrychlení. To uděláme v následující části Eulerovou metodou.


### Odpor prostředí

Lineární odpor vždy působí proti relativní rychlosti $\mathbf{v} - \mathbf{w}$. Pokud je $\mathbf{w}$ konstantní, rychlost se asymptoticky blíží $\mathbf{w}$, tedy
$$
\mathbf{v}_\infty = \mathbf{w}.
$$
To uvidíme na grafu velikosti rychlosti i na trajektorii.


---


## Numerický výpočet rychlosti ze zrychlení – Eulerova metoda

Když už máme model zrychlení $\mathbf{a}(\mathbf{v}, \mathbf{r}, t)$ (např. $\mathbf{a} = (c/m)(\mathbf{w}-\mathbf{v})$), můžeme rychlost počítat krok za krokem. Explicitní Eulerova metoda je nejjednodušší schéma:
$$
\mathbf{v}_{i} = \mathbf{v}_{i-1} + \mathbf{a}(\mathbf{v}_{i-1}, \mathbf{r}_{i-1}, t_{i-1})\,\Delta t.
$$
Metoda je přímočará, ale citlivá na velikost dt; příliš velký krok vede k chybám i nestabilitě. Ilustraci zde nemáme, proto si vystačíme se vzorcem a pečlivým výběrem kroku.


---


## Úkol 2: Rychlost ze zrychlení

Použijeme nejjednodušší případ odporu vůči konstantnímu $\mathbf{w}$, tedy $\mathbf{w} = \mathrm{konst}$. Zrychlení není konstantní, protože závisí na rychlosti $\mathbf{v}$, ale Eulerovu metodu můžeme použít přímo.

Zadání: napište Eulerův krok pro rychlost a ověřte jej na konstantním $\mathbf{w}$. Trajektorii pak dopočítejte pomocí funkce z Úkolu 1. Poznámka: r zde slouží jen kvůli podpisu a_funkce, skutečnou trajektorii počítáme zvlášť.


In [ ]:
def euler_rychlost_2d(v0, a_funkce, t, dt):
    """
    Explicitní Euler pro rychlost v 2D.

    Parametry:
        v0       : počáteční rychlost [m/s], pole tvaru (2,)
        a_funkce : funkce a(v, r, t) -> (ax, ay)
        t        : 1D pole časů [s]
        dt       : krok času [s]

    Návrat:
        a, v : zrychlení a rychlost v čase
    """
    t = np.asarray(t, dtype=float)
    n = t.size
    v = np.zeros((n, 2), dtype=float)
    a = np.zeros((n, 2), dtype=float)
    r = np.zeros((n, 2), dtype=float)
    v[0] = np.asarray(v0, dtype=float)

    ###### doplňte kód zde ######
    a[0] = a_funkce(v[0], r[0], t[0])
    for i in range(1, n):
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
        a[i] = a_funkce(v[i], r[i], t[i])
    #############################

    return a, v


# Jednoduchý test: odpor vůči konstantnímu $w$
m_test = 2.0  # kg
c_test = 1.2  # kg/s
w_test = np.array([1.5, 0.3])  # m/s
v0 = np.array([0.0, 0.0])

t0, t1, dt = 0.0, 8.0, 0.05
t = np.arange(t0, t1 + dt, dt)


def a_const(v, r, t):
    return (c_test / m_test) * (w_test - v)


a_euler, v_euler = euler_rychlost_2d(v0, a_const, t, dt)


In [ ]:
# Trajektorie z vypočtené rychlosti
r0 = np.array([0.0, 0.0])
r_euler = poloha_2d(r0, v_euler, dt)

vykresli_drift(t, r_euler, v_euler, title="Úkol 2: Euler pro rychlost")


---


## Úkol 3: Drift s konstantní rychlostí prostředí a odporem

Nyní spojíme Euler pro rychlost a lichoběžníkové pravidlo pro polohu do jedné simulace. V této úloze už skutečně potřebujeme r, protože rychlost prostředí může být funkcí polohy.

Použijte parametry a jednotky uvedené v kódu a sledujte, jak se rychlost přibližuje k $\mathbf{w}$.


In [ ]:
# Parametry systému
x0, y0 = 0.0, 0.0
r0 = np.array([x0, y0])

v0 = np.array([0.0, 0.0])
t0 = 0.0
t1 = 60.0
dt = 0.05

m = 5.0   # kg
c = 2.0   # kg/s
w = np.array([1.2, 0.4])  # m/s

t = np.arange(t0, t1 + dt, dt)


def simuluj_drift(r0, v0, t, dt, a_funkce):
    """
    Simulace driftu: Euler pro rychlost a lichoběžník pro polohu.
    """
    t = np.asarray(t, dtype=float)
    n = t.size
    r = np.zeros((n, 2), dtype=float)
    v = np.zeros((n, 2), dtype=float)
    a = np.zeros((n, 2), dtype=float)

    r[0] = np.asarray(r0, dtype=float)
    v[0] = np.asarray(v0, dtype=float)
    a[0] = a_funkce(v[0], r[0], t[0])

    for i in range(1, n):
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
        a[i] = a_funkce(v[i], r[i], t[i])

    return r, v, a


def w_ext_const(r, t):
    """
    Konstantní rychlost prostředí $w$.
    """
    return w


def a_drift(v, r, t):
    """
    Zrychlení pro drift s odporem vůči prostředí.
    """
    a = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    a = (c / m) * (w_ext_const(r, t) - v)
    #############################
    return a


r_drift, v_drift, a_drift_hist = simuluj_drift(r0, v0, t, dt, a_drift)

vykresli_drift(t, r_drift, v_drift, title="Úkol 3: drift s odporem vůči prostředí")


Pozorování: Trajektorie se nejprve rychle rozbíhá, ale velikost rychlosti se postupně stabilizuje. To odpovídá přiblížení k limitní rychlosti |v_inf| = |F|/c, protože odpor začne vyrovnávat vnější sílu. Z grafu |v(t)| lze vidět, že k této hodnotě směřujeme hladce a bez oscilací.


---


## Úkol 4: A co když se rychlost prostředí mění v prostoru?

V reálné situaci se rychlost prostředí může měnit s polohou (např. rychlostní pole kolem překážky). Zavedeme jednoduché pole $\mathbf{w}(\mathbf{r}, t)$, které částici „táhne“ směrem k bodu r*.

Tím vznikne zakřivená trajektorie a uvidíte, že stejné numerické schéma funguje i pro složitější pole. Stejný zápis dovoluje i časovou změnu pole, stačí do $\mathbf{w}$ přidat závislost na t. Na co si dát pozor: zvolte takový krok dt, aby trajektorie zůstala hladká.


In [ ]:
k = 1.5e-4  # 1/s
r_star = np.array([10_000.0, 5_000.0])


def w_ext_pole(r, t):
    """
    Příklad pole $w(\mathbf{r},t)$: rychlost prostředí směřuje k bodu r_star.
    """
    w_loc = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    w_loc = k * (r_star - r)
    #############################
    return w_loc


def a_drift_pole(v, r, t):
    return (c / m) * (w_ext_pole(r, t) - v)


r_pole, v_pole, a_pole = simuluj_drift(r0, v0, t, dt, a_drift_pole)

vykresli_drift(t, r_pole, v_pole, title="Úkol 4: drift v poli rychlosti")


---


## Ukázka: časově proměnné vektorové pole rychlosti prostředí (syntetická data)

V této části vytvoříme jednoduché, ale realisticky se měnící pole rychlosti prostředí na mřížce. Budeme pracovat s oblastí 100 x 100 km a 30 denními snímky, kde je pole v každé buňce mřížky konstantní. Vektorové pole $\mathbf{w}(\mathbf{r}, t)$ chápeme jako rychlost prostředí [m/s].

Postup:
- vygenerujeme časovou řadu polí na mřížce,
- vizualizujeme několik časových snímků pomocí šipek,
- spočteme trajektorii v čase a vykreslíme ji.


In [ ]:
# Generování syntetického pole na mřížce (větší oblast, bez dlouhodobě preferovaného směru)
L = 2_000_000.0  # m (1200 km => -600 .. 600 km)
nx, ny = 21, 21
x = np.linspace(-0.5 * L, 0.5 * L, nx)
y = np.linspace(-0.5 * L, 0.5 * L, ny)
xx, yy = np.meshgrid(x, y)

# Normalizované souřadnice v intervalu [-1, 1]
X = xx / (0.5 * L)
Y = yy / (0.5 * L)

# Časová osa: 30 dní po 1 dni
DAY = 24 * 3600.0
n_days = 30
t_grid = np.arange(0, n_days + 1) * DAY
nt = t_grid.size

W_grid = np.zeros((nt, ny, nx, 2), dtype=float)

for k, tk in enumerate(t_grid):
    # uniformní složka, která se v čase plynule otáčí (bez preferovaného směru)
    theta = 2.0 * np.pi * tk / (5.0 * DAY)
    base_amp = 2.2 + 0.6 * np.sin(2.0 * np.pi * tk / (3.0 * DAY))
    base = base_amp * np.array([np.cos(theta), np.sin(theta)])

    # střídavě radiální „pumpa“ (dovnitř/ven)
    radial_amp = 1.2 * np.sin(2.0 * np.pi * tk / (2.2 * DAY))
    radial = radial_amp * np.stack([X, Y], axis=-1) * np.exp(-0.8 * (X**2 + Y**2))[..., None]

    # dvojice protisměrných vírů s centry na kružnici
    cx = 0.45 * np.cos(2.0 * np.pi * tk / (6.0 * DAY))
    cy = 0.45 * np.sin(2.0 * np.pi * tk / (6.0 * DAY))
    Xc = X - cx
    Yc = Y - cy
    vortex1 = 5.0 * np.stack([-Yc, Xc], axis=-1) * np.exp(-4.0 * (Xc**2 + Yc**2))[..., None]

    Xc2 = X + cx
    Yc2 = Y + cy
    vortex2 = -5.0 * np.stack([-Yc2, Xc2], axis=-1) * np.exp(-4.0 * (Xc2**2 + Yc2**2))[..., None]

    # vlnové složky putující v čase
    phase1 = 2.0 * np.pi * (0.7 * X + 0.4 * Y + tk / (4.5 * DAY))
    phase2 = 2.0 * np.pi * (0.3 * X - 0.8 * Y - tk / (3.2 * DAY))
    wave1 = 1.8 * np.stack([np.cos(phase1), np.sin(phase1)], axis=-1)
    wave2 = 1.2 * np.stack([-np.sin(phase2), np.cos(phase2)], axis=-1)

    W_grid[k] = base + radial + vortex1 + vortex2 + wave1 + 0.7 * wave2


In [ ]:
# Hourly animation (shorter arrows, saved to mp4 and embedded)
XX, YY = np.meshgrid(x / 1000.0, y / 1000.0)
HOUR = 3600.0

t_anim = np.arange(t_grid[0], t_grid[-1] + HOUR, HOUR)
W_anim = np.empty((t_anim.size, ny, nx, 2), dtype=float)

for i, t in enumerate(t_anim):
    k0 = int(np.clip(np.searchsorted(t_grid, t) - 1, 0, nt - 2))
    alpha = (t - t_grid[k0]) / (t_grid[k0 + 1] - t_grid[k0])
    W_anim[i] = (1.0 - alpha) * W_grid[k0] + alpha * W_grid[k0 + 1]

stride = 2  # subsample arrows for readability and smaller output
XXs = XX[::stride, ::stride]
YYs = YY[::stride, ::stride]

fig, ax = plt.subplots(figsize=(6, 5), dpi=90)
U0 = W_anim[0, ::stride, ::stride, 0]
V0 = W_anim[0, ::stride, ::stride, 1]
quiv = ax.quiver(XXs, YYs, U0, V0, scale=40.0, width=0.0025, pivot='mid')
title = ax.set_title(f"t = {t_anim[0] / HOUR:.0f} h")
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
ax.set_aspect('equal')
ax.set_xlim(x.min() / 1000.0, x.max() / 1000.0)
ax.set_ylim(y.min() / 1000.0, y.max() / 1000.0)
ax.grid(True, alpha=0.3)

from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import Video


def update(frame):
    U = W_anim[frame, ::stride, ::stride, 0]
    V = W_anim[frame, ::stride, ::stride, 1]
    quiv.set_UVC(U, V)
    elapsed = t_anim[frame]
    days = int(elapsed // DAY)
    hours = int((elapsed % DAY) // HOUR)
    title.set_text(f"t = {days} d {hours:02d} h")
    return quiv, title


anim = FuncAnimation(fig, update, frames=t_anim.size, interval=120, blit=False)

out_path = "vector_field_hourly.mp4"
writer = FFMpegWriter(fps=8, bitrate=1800)
anim.save(out_path, writer=writer, dpi=90)
plt.close(fig)

Video(out_path, embed=True)


In [ ]:
# Trajektorie v časově proměnném poli
nx = x.size
ny = y.size
nt = t_grid.size

x0_grid = x[0]
y0_grid = y[0]
dx = x[1] - x[0]
dy = y[1] - y[0]

t0_field = t_grid[0]
dt_field = t_grid[1] - t_grid[0]


def w_ext_grid(r, t):
    """
    Rychlost prostředí w z časově proměnného pole na mřížce (konstantní v buňce).
    """
    if not np.all(np.isfinite(r)) or not np.isfinite(t):
        return np.zeros(2, dtype=float)
    ix = int(np.clip(np.floor((r[0] - x0_grid) / dx), 0, nx - 1))
    iy = int(np.clip(np.floor((r[1] - y0_grid) / dy), 0, ny - 1))
    it = int(np.clip(np.floor((t - t0_field) / dt_field), 0, nt - 1))
    return W_grid[it, iy, ix]


# Parametry simulace (oddeleno od predchozich ukolu)
r0_field = np.array([0.0, 0.0])  # start uprostred oblasti
v0_field = np.array([0.0, 0.0])

m_field = 5.0   # kg
c_field = 0.02  # kg/s (vetsi casova konstanta)

t0_field = 0.0
t1_field = 7.0 * DAY

dt_sim = 30.0  # 30 s

t_field = np.arange(t0_field, t1_field + dt_sim, dt_sim)


def a_drift_cas(v, r, t):
    a = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    a = (c_field / m_field) * (w_ext_grid(r, t) - v)
    #############################
    return a


r_td, v_td, a_td = simuluj_drift(r0_field, v0_field, t_field, dt_sim, a_drift_cas)

vykresli_drift(t_field, r_td, v_td, title='Drift v časově proměnném poli')


---


## Vliv hmotnosti a odporu na trajektorii

Porovnáme několik typů trosek se stejným startem a stejnou délkou simulace. Měníme hmotnost m [kg] a koeficient odporu c [kg/s], což modeluje různé "tvary" a velikosti trosek. Fyzikálně platí, že větší odpor rychleji tlumí relativní pohyb vůči $\mathbf{w}$, zatímco větší hmotnost reaguje pomaleji na změny rychlosti prostředí.


In [ ]:
# Porovnani ruznych m a c pro stejný start a stejný čas
parametry = [
    ("lehká + malý odpor", 2.0, 0.01),
    ("lehká + velký odpor", 2.0, 0.001),
    ("těžká + malý odpor", 120.0, 0.01),
    ("těžká + velký odpor", 120.0, 0.001),
]

fig, ax = plt.subplots(figsize=(6, 6))

for label, m_cmp, c_cmp in parametry:
    def a_drift_cmp(v, r, t, m=m_cmp, c=c_cmp):
        return (c / m) * (w_ext_grid(r, t) - v)

    r_cmp, v_cmp, _ = simuluj_drift(r0_field, v0_field, t_field, dt_sim, a_drift_cmp)
    ax.plot(r_cmp[:, 0] / 1000.0, r_cmp[:, 1] / 1000.0, label=f"{label} (m={m_cmp}, c={c_cmp})")

ax.scatter(r0_field[0] / 1000.0, r0_field[1] / 1000.0, color='black', s=20, label='start')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
ax.set_title('Vliv m a c na trajektorii v časově proměnném poli')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=8)
plt.tight_layout()
plt.show()


---


## Závěr a rychlá funkce pro výslednou polohu

Pro konstantní rychlost prostředí $\mathbf{w}$ a nulovou počáteční rychlost existuje uzavřené řešení bez smyček. Nejprve si označme
$$
\Delta t = t_1 - t_0, \qquad \tau = \frac{m}{c},
$$
potom platí
$$
\mathbf{r}(t_1)=\mathbf{r}_0+\mathbf{w}\,\Delta t - \tau\,\mathbf{w}\left(1-e^{-\Delta t/\tau}\right).
$$
Tento vztah je numericky stabilní i pro malé kroky, pokud použijeme funkci expm1. To je praktické, když potřebujeme jen konečnou polohu.


In [ ]:
def final_position_constant_wind(x0, y0, t0, t1, m, c, wx, wy):
    """
    Analytická poloha pro konstantní rychlost prostředí a v0 = 0.

    Parametry:
        x0, y0 : počáteční poloha [m]
        t0, t1 : čas [s]
        m      : hmotnost [kg]
        c      : koeficient odporu [kg/s]
        wx, wy : složky rychlosti prostředí [m/s]

    Návrat:
        (x1, y1) : poloha v čase t1 [m]
    """
    dt = t1 - t0
    r0 = np.array([x0, y0], dtype=float)
    w = np.array([wx, wy], dtype=float)
    r1 = r0.copy()

    ###### doplňte kód zde ######
    if c == 0:
        r1 = r0
    else:
        tau = m / c
        term = np.expm1(-dt / tau)
        r1 = r0 + w * dt + tau * w * term
    #############################

    return float(r1[0]), float(r1[1])


# Validace: numerika vs. analytika z Úkolu 3
r_fast = np.array(final_position_constant_wind(x0, y0, t0, t1, m, c, w[0], w[1]))
r_num = r_drift[-1]

print("numerika:", r_num)
print("analyticky:", r_fast)
print("allclose:", np.allclose(r_num, r_fast, rtol=1e-3, atol=1e-3))


---


## Shrnutí

- Umím formálně popsat $\mathbf{r}(t)$, $\mathbf{v}(t)$ a $\mathbf{a}(t)$ v 2D.
- Chápu, že jediná síla je odpor závislý na relativní rychlosti $\mathbf{v} - \mathbf{w}$.
- Vím, proč kombinujeme Eulerův krok pro $\mathbf{v}$ a lichoběžník pro $\mathbf{r}$.
- Umím volit rozumný krok dt a vím, že příliš velké dt zhorší přesnost.
- Rozumím významu m, c a pole $\mathbf{w}$ a jejich vlivu na přizpůsobení rychlosti.
- Typické chyby: špatné jednotky, zapomenutá prealokace, pole špatného tvaru (n, 2).
